In [ ]:
import os
import numpy as np
import librosa
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Dropout, Flatten, Dense
from tensorflow.keras.utils import to_categorical

In [4]:
def extract_mfcc(file_path, n_mfcc=40, max_len=173):
    y, sr = librosa.load(file_path, sr=22050)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    if mfcc.shape[1] < max_len:
        pad_width = max_len - mfcc.shape[1]
        mfcc = np.pad(mfcc, ((0, 0), (0, pad_width)), mode='constant')
    else:
        mfcc = mfcc[:, :max_len]
    return mfcc.T  # (173, 40)


In [ ]:
data_dir = "dataset"  

print("Files:", os.listdir(data_dir))

X, y = [], []

for label in os.listdir(data_dir):
    folder = os.path.join(data_dir, label)
    if not os.path.isdir(folder):
        continue
    print("Processing:", label)
    for file in os.listdir(folder):
        if file.endswith(".wav") or file.endswith(".mp3"):
            path = os.path.join(folder, file)
            try:
                features = extract_mfcc(path)
                X.append(features)
                y.append(label)
            except Exception as e:
                print("Error:", path, e)


In [ ]:
X = np.array(X)

le = LabelEncoder()
y_encoded = le.fit_transform(y)
y_cat = to_categorical(y_encoded)

print("X shape:", X.shape)   # (örnek_sayısı, 173, 40)
print("y shape:", y_cat.shape)
print("Labels:", le.classes_)


In [ ]:
num_classes = y_cat.shape[1]

model = Sequential([
    Conv1D(32, 3, activation='relu', input_shape=(173, 40)),
    MaxPooling1D(2),
    Dropout(0.3),
    Conv1D(64, 3, activation='relu'),
    MaxPooling1D(2),
    Dropout(0.3),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y_cat, test_size=0.2, random_state=42)

history = model.fit(
    X_train, y_train,
    epochs=100,
    batch_size=16,
    validation_data=(X_test, y_test)
)


In [ ]:
model.save("mfcc_cnn_model_cat.h5")

import joblib
joblib.dump(le, "label_encoder_cat.pkl")


In [ ]:
def analyze_predictions(preds, labels):
    # Tüm sınıflar
    probs = {labels[i]: float(preds[i]) for i in range(len(preds))}
    
    # En yüksekten sırala
    sorted_probs = sorted(probs.items(), key=lambda x: x[1], reverse=True)
    
    # Senaryo analizi
    top1, top1_val = sorted_probs[0]
    top2, top2_val = sorted_probs[1]
    
    if top1_val >= 0.7:
        case = "single"
        focus = {top1: top1_val}
    elif (top1_val + top2_val) >= 0.7:
        case = "double"
        focus = {top1: top1_val, top2: top2_val}
    else:
        case = "mixed"
        focus = dict(sorted_probs[:3])  # en iyi 3
    
    return probs, case, focus


In [ ]:
label, confidence = analyze_predictions("example.mp3")
print("Prediction:", label, "Confidince Scroe:", confidence)
